# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df.copy()
X.drop(["loan_status"], axis=1, inplace=True)
X.head()
X = pd.get_dummies(X)

# Create our target
y = df["loan_status"].values.reshape(-1, 1)

In [6]:
# Check the balance of our target values
y

array([['low_risk'],
       ['low_risk'],
       ['low_risk'],
       ...,
       ['low_risk'],
       ['low_risk'],
       ['low_risk']], dtype=object)

In [7]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [8]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
scaler = StandardScaler()

In [9]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
# When fitting scaling functions, only train on the training dataset

In [10]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [11]:
# Resample the training data with the BalancedRandomForestClassifier
#balanced is undersampling
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)
y_pred = brf.predict(X_test_scaled)

In [12]:
# Display the confusion matrix
y_pred = brf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   43,    44],
       [ 6296, 10822]])

In [13]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.5632264484652171

In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.49      0.63      0.01      0.56      0.31        87
   low_risk       1.00      0.63      0.49      0.77      0.56      0.32     17118

avg / total       0.99      0.63      0.49      0.77      0.56      0.32     17205



In [15]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_ 
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print(f"{X.columns.values[indices[f]]}: ({importances[indices[f]]})")

total_rec_prncp: (0.07376667607601396)
total_rec_int: (0.06390324452717588)
total_pymnt_inv: (0.06073336071656837)
total_pymnt: (0.05811195697921674)
last_pymnt_amnt: (0.04951778391272079)
int_rate: (0.02458051789018817)
out_prncp: (0.020398879691407974)
dti: (0.018625883307901298)
max_bal_bc: (0.018378884967316288)
issue_d_Jan-2019: (0.017480030880564042)
installment: (0.01698703048268034)
annual_inc: (0.01661321625916749)
bc_util: (0.01623198470200134)
mths_since_recent_inq: (0.015821923028819504)
total_bc_limit: (0.015385686591272854)
out_prncp_inv: (0.015337070167127763)
avg_cur_bal: (0.01506649550412446)
issue_d_Mar-2019: (0.014926455495755147)
revol_bal: (0.014924489307566175)
mo_sin_old_rev_tl_op: (0.014583210727445232)
total_bal_ex_mort: (0.01428231366275729)
tot_cur_bal: (0.014238473969302518)
mo_sin_old_il_acct: (0.014067369111282934)
total_bal_il: (0.01351439379612201)
bc_open_to_buy: (0.013343855077273033)
il_util: (0.013138998568932102)
tot_hi_cred_lim: (0.0131060052487096

### Easy Ensemble Classifier

In [16]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [17]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9254565671948463

In [18]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred)

array([[   79,     8],
       [  978, 16140]])

In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [21]:
#1. Easy Ensemble Classifier has the best balanced accuracy score at 0.9254565671948463
#2. Easy Ensemble Classifier has the best recall score at .94
#3. Easy Ensemble Classifier has the best geometric at .93
'''#4. Precision, recall and Support. Precision is important because data is useless if it's not accurate. 
Recall is important because if we do not catch the majority of the positive cases then the model used is insignificant. 
Lastly the Support is important because without a large enough dataset, running models will be difficult to rely on.'''

"#4. Precision, recall and Support. Precision is important because data is useless if it's not accurate. \nRecall is important because if we do not catch the majority of the positive cases then the model used is insignificant. \nLastly the Support is important because without a large enough dataset, running models will be difficult to rely on."